In [1]:
from nba_api.stats.endpoints import commonallplayers
import pandas as pd
import sqlite3

## Obtain Current Player Information

In [2]:
# get common info of current players
cp = commonallplayers.CommonAllPlayers(season='2023-24')

# obtain the first data frame
cp_df = cp.get_data_frames()[0]

# subset by players that have been playing since 2020.
curr_players_info = cp_df[cp_df['TO_YEAR'] >= '2020']

In [3]:
curr_players_info.to_csv('./intermediate_data/curr_players_info.csv', header = True, index=False)

## SQL Connection: Join Contracts and Player DF in SQL DB

### Load Data

In [4]:
nba_contracts = pd.read_csv('./raw_data/nba_contracts.csv')

### Transform Data

In [5]:
# obtain a better ID to merge on
nba_contracts['player_slug'] = nba_contracts['FreeAgent'].str.lower()
nba_contracts['player_slug'] = nba_contracts['player_slug'].str.replace(' ', '_', regex=False).str.replace('.', '', regex=False).str.replace("'", '', regex=False)

In [6]:
sqliteConnection = sqlite3.connect('nba_data.db')

# write current player info and contract data as SQL tables
curr_players_info.to_sql(name = 'curr_players_info', con = sqliteConnection, index = True, if_exists='replace')
nba_contracts.to_sql(name = 'nba_contracts', con = sqliteConnection, index=True, if_exists='replace')

sqliteConnection.close()

### SQL Query to merge the 2 datasets

In [7]:
sqliteConnection = sqlite3.connect('nba_data.db')
# Create cursor object 
cursor = sqliteConnection.cursor() 
  
# Query for INNER JOIN 
sql = '''SELECT C.PERSON_ID, N.FreeAgent, N.Position, N.AVG_SALARY, N.FA_Year 
FROM nba_contracts N
LEFT JOIN curr_players_info C
ON N.player_slug = C.PLAYER_SLUG;'''
  
# Executing the query 
cursor.execute(sql) 
  
# Fetching rows from the result table 
result = cursor.fetchall() 

person_id_df = pd.DataFrame(result, columns = ['ID', 'Name', 'Position', 'AVG_SALARY', 'FA_Year'])
  
# Closing the connection 
sqliteConnection.close() 

In [8]:
# obtaining the ids of free Agents to be connected to their statistics
fa_ids = person_id_df[person_id_df['ID'] == person_id_df['ID']]

# transform data to include previous season
fa_ids = fa_ids.sort_values(by = ['ID', 'FA_Year'])
fa_ids.loc[:, 'Prev_Season'] = fa_ids.apply(lambda x: str(x['FA_Year']-1) + '-' + str(x['FA_Year'])[2:], axis = 1)

### Load Cleaned Data to SQL Database

In [9]:
sqliteConnection = sqlite3.connect('./nba_data.db')

fa_ids.to_sql(name = 'freeAgentInfo', con = sqliteConnection, index = True, if_exists='replace')

sqliteConnection.close()

## NBA Free Agent Stats

### Extract Data

In [10]:
from nba_api.stats.endpoints import playercareerstats
import time

# For every single free agent, obtain their statistics from their previous year.
items = []
for index, row in fa_ids.iterrows():
    playerID = int(row['ID'])
    seasonID = str(row['FA_Year']-1) + '-' + str(row['FA_Year'])[2:]
    playerStats = playercareerstats.PlayerCareerStats(playerID)
    playerStats_DF = playerStats.season_totals_regular_season.get_data_frame()
    playerStats_row = playerStats_DF[playerStats_DF['SEASON_ID'] == seasonID].to_dict(orient = "records")
    if playerStats_row:
        playerStats_row = playerStats_row[-1]
        items.append(playerStats_row)
    time.sleep(1)
    

### Load Data

In [12]:
sqliteConnection = sqlite3.connect('nba_data.db')
nba_FA_szn_stats = pd.DataFrame(items)
nba_FA_szn_stats.to_sql(name = 'freeAgentStats', con = sqliteConnection, index = True, if_exists='replace')

sqliteConnection.close()

### SQL Connection: Connect FAs with their Stats

In [13]:
sqliteConnection = sqlite3.connect('nba_data.db')
# Create cursor object 
cursor = sqliteConnection.cursor() 
  
# Query for INNER JOIN 
sql = '''SELECT FAS.PLAYER_ID, FAI.Name, FAI.Position, FAI.AVG_SALARY, FAI.FA_Year, FAS.PLAYER_AGE, FAS.GP, FAS.MIN, FAS.FGM,
FAS.FGA, FAS.FG_PCT, FAS.FG3M, FAS.FG3A, FAS.FG3_PCT, FAS.FTM, FAS.FTA, FAS.FT_PCT, FAS.OREB, FAS.DREB,
FAS.REB, FAS.AST, FAS.STL, FAS.BLK, FAS.TOV, FAS.PF, FAS.PTS
FROM freeAgentInfo FAI
LEFT JOIN freeAgentStats FAS
ON FAI.ID = FAS.PLAYER_ID
AND FAI.Prev_Season = FAS.SEASON_ID;'''
  
# Executing the query 
cursor.execute(sql) 
  
# Fetching rows from the result table 
result = cursor.fetchall() 

fa_full = pd.DataFrame(result, columns = ['ID', 'Name', 'Position', 'Contract', 'FA_Year', 'Age',
                                            'GP', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A',
                                            'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB',
                                            'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS'])
  
# Closing the connection 
sqliteConnection.close() 

### SQL Connection: Connect FAs with Stats with All-Star and All-NBA

In [14]:
sqliteConnection = sqlite3.connect('nba_data.db')
# Create cursor object 
cursor = sqliteConnection.cursor() 
  
# Query for INNER JOIN 
sql = '''SELECT FAS.PLAYER_ID, FAI.Name, FAI.Position, FAI.AVG_SALARY, FAI.FA_Year, FAS.PLAYER_AGE, FAS.GP, FAS.MIN, FAS.FGM,
FAS.FGA, FAS.FG_PCT, FAS.FG3M, FAS.FG3A, FAS.FG3_PCT, FAS.FTM, FAS.FTA, FAS.FT_PCT, FAS.OREB, FAS.DREB,
FAS.REB, FAS.AST, FAS.STL, FAS.BLK, FAS.TOV, FAS.PF, FAS.PTS, aN.is_AllNBA, aST.isAllStar
FROM freeAgentInfo FAI
LEFT JOIN freeAgentStats FAS
ON FAI.ID = FAS.PLAYER_ID
AND FAI.Prev_Season = FAS.SEASON_ID
LEFT JOIN allNBA aN
ON FAI.Name = aN.Name
AND FAI.Prev_Season = aN.Season
LEFT JOIN allstar aST
ON FAI.Name = aST.Name
AND FAI.Prev_Season = aST.Season;'''
  
# Executing the query 
cursor.execute(sql) 
  
# Fetching rows from the result table 
result = cursor.fetchall() 

fa_full_ALLNBA = pd.DataFrame(result, columns = ['ID', 'Name', 'Position', 'Contract', 'FA_Year', 'Age',
                                            'GP', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A',
                                            'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB',
                                            'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'is_AllNBA', 'is_AllStar'])

# Data Transformation and Relabeling
fa_full_ALLNBA['is_AllNBA'] = fa_full_ALLNBA['is_AllNBA'].fillna(value=0)
fa_full_ALLNBA['is_AllStar'] = fa_full_ALLNBA['is_AllStar'].fillna(value=0)
fa_full_ALLNBA['PPG'] = fa_full_ALLNBA['PTS']/fa_full_ALLNBA['GP']
fa_full_ALLNBA['APG'] = fa_full_ALLNBA['AST']/fa_full_ALLNBA['GP']
fa_full_ALLNBA['RPG'] = fa_full_ALLNBA['REB']/fa_full_ALLNBA['GP']
fa_full_ALLNBA['SPG'] = fa_full_ALLNBA['STL']/fa_full_ALLNBA['GP']
fa_full_ALLNBA['BPG'] = fa_full_ALLNBA['BLK']/fa_full_ALLNBA['GP']
fa_full_ALLNBA['MPG'] = fa_full_ALLNBA['MIN']/fa_full_ALLNBA['GP']
fa_full_ALLNBA['eFG'] = (fa_full_ALLNBA['FGM'] + fa_full_ALLNBA['FG3M']*0.5)/fa_full_ALLNBA['FGA']
fa_full = fa_full_ALLNBA[['Name', 'ID', 'Contract', 'FA_Year', 'Age', 'eFG', 'PPG', 'APG', 'RPG', 'SPG', 'BPG', 'MPG', 'is_AllNBA', 'is_AllStar']]

fa_full.to_sql(name = 'FA_FULL', con = sqliteConnection, index=True, if_exists = 'replace')

# Closing the connection 
sqliteConnection.close() 